In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider
%matplotlib widget


# --------------------------
# ПАРАМЕТРЫ МАНИПУЛЯТОРА
# --------------------------
l1 = 1.0   # расстояние от 1 до 2 узла
l3 = 0.5   # длина "хвата" от узла до инструмента



In [2]:
def set_axes_equal(ax):
    """Сделать масштабирование 3D-оси равным (одинаковая длина по X,Y,Z)."""
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    y_range = abs(y_limits[1] - y_limits[0])
    z_range = abs(z_limits[1] - z_limits[0])

    max_range = max([x_range, y_range, z_range]) / 2.0

    mid_x = np.mean(x_limits)
    mid_y = np.mean(y_limits)
    mid_z = np.mean(z_limits)

    ax.set_xlim3d([mid_x - max_range, mid_x + max_range])
    ax.set_ylim3d([mid_y - max_range, mid_y + max_range])
    ax.set_zlim3d([mid_z - max_range, mid_z + max_range])

In [3]:
# --------------------------
# МАТРИЦЫ ПРЕОБРАЗОВАНИЙ
# --------------------------
def A1(q1):
    return np.array([
        [1, 0, 0, l1],
        [0, 0, 1, 0],
        [0,-1, 0, q1],
        [0, 0, 0, 1]
    ])

def A2(q2, q1):
    return np.array([
        [1, 0, 0, l1],
        [0, 1, 0, q2],
        [0, 0, 1, q1],
        [0, 0, 0, 1]
    ])

def A3(q3, q2, q1):
    """Конечное преобразование (из первой матрицы твоего диалога)."""
    return np.array([
        [np.cos(q3), -np.sin(q3), 0, l1 + l3*np.cos(q3)],
        [np.sin(q3),  np.cos(q3), 0, q2 + l3*np.sin(q3)],
        [0,           0,          1, q1],
        [0,           0,          0, 1]
    ])

In [4]:
def reverce_kinematic(x_abs, y_abs, z_abs):
    
    x = z_abs
    y = x_abs
    z = y_abs
    
    q1 = z
    
    s = x - l1
    
    if (l3**2 < s**2):
        print("ERRRORRR")
    
    rho = np.sqrt(l3**2 - s**2) 
    
    if y < 0:
        rho = -rho
        
    
        
        
    q3 = np.arctan2(rho, s)
    q2 = y - rho
    
    
    return q1, q2, q3
    
    

In [5]:
# --------------------------
# ПЕРЕВОД В СИСТЕМУ ПОЛЬЗОВАТЕЛЯ
def Go2abs(v):
    return v[1], v[2], v[0]


# --------------------------
# ПРЯМАЯ КИНЕМАТИКА
# --------------------------
def forward_kinematics(q1, q2, q3):
    # последовательное умножение матриц
    T1 = A1(q1)
    T2 = A2(q2, q1)
    T3 = A3(q3, q2, q1)

    # Позиции всех узлов
    p0 = np.array([0,0,0,1])
    p1 = Go2abs(T1 @ np.array([0,0,0,1]))  # Узел 1
    p2 = Go2abs(T2 @ np.array([0,0,0,1]))  # Узел 2
    p3 = Go2abs(T3 @ np.array([0,0,0,1]))  # хват (Узел 3)

    return p0[:3], p1[:3], p2[:3], p3[:3], T3

In [6]:
# --- Рисовалка манипулятора ---
def plot_manipulator(x=0, y=0, z=0):
    
    q1, q2, q3 = reverce_kinematic(x, y, z)
    
    p0, p1, p2, p3, T = forward_kinematics(q1, q2, q3)


    '''
    # Создаем фигуру с большой левой колонкой для 3D и правой для проекций
    fig = plt.figure(figsize=(15, 6))
    gs = fig.add_gridspec(2, 4, width_ratios=[3,1,1,1], height_ratios=[1,1], wspace=0.7, hspace=0.3)
    # ---- 3D-график (левая большая область) ----
    ax1 = fig.add_subplot(gs[:,0], projection='3d')
    '''
    
    
    # Создаем фигуру с большой левой колонкой для 3D и правой для проекций
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.add_subplot(111, projection='3d')

    
    # Линии звеньев
    ax1.plot([p0[0], p1[0]], [p0[1], p1[1]], [p0[2], p1[2]], 'ro-')
    ax1.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'go-')
    ax1.plot([p2[0], p3[0]], [p2[1], p3[1]], [p2[2], p3[2]], 'bo-')

    
    
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("Z")

    set_axes_equal(ax1)
    #plt.show()
    '''   
    # ---- XY проекция (вид сверху) ----
    ax2 = fig.add_subplot(gs[0,1])

    # Линии звеньев
    ax2.plot([p0[0], p1[0]], [p0[1], p1[1]], 'ro-')
    ax2.plot([p1[0], p2[0]], [p1[1], p2[1]], 'go-')
    ax2.plot([p2[0], p3[0]], [p2[1], p3[1]], 'bo-')
    ax2.set_title("Проекция XY (вид сверху)")
    ax2.set_xlabel("X"); ax2.set_ylabel("Y")
    ax2.axis('equal')

    # ---- XZ проекция (вид сбоку) ----
    ax3 = fig.add_subplot(gs[0,2])
        # Линии звеньев
    ax3.plot([p0[0], p1[0]],  [p0[2], p1[2]], 'ro-')
    ax3.plot([p1[0], p2[0]],  [p1[2], p2[2]], 'go-')
    ax3.plot([p2[0], p3[0]],  [p2[2], p3[2]], 'bo-')
    ax3.set_title("Проекция XZ (вид сбоку)")
    ax3.set_xlabel("X"); ax3.set_ylabel("Z")
    ax3.axis('equal')

    # ---- YZ проекция (вид спереди) ----
    ax4 = fig.add_subplot(gs[0,3])
        # Линии звеньев
    ax4.plot([p0[1], p1[1]], [p0[2], p1[2]], 'ro-')
    ax4.plot([p1[1], p2[1]], [p1[2], p2[2]], 'go-')
    ax4.plot([p2[1], p3[1]], [p2[2], p3[2]], 'bo-')
    ax4.set_title("Проекция YZ (вид спереди)")
    ax4.set_xlabel("Y"); ax4.set_ylabel("Z")
    ax4.axis('equal')

    # Отступы между графиками
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    
    ''' 

    plt.show()

In [7]:

# --- Интерактив ---
interact(plot_manipulator,
         x=FloatSlider(min=-2, max=2, step=0.1, value=0),
         y=FloatSlider(min=-2, max=2, step=0.1, value=0),
         z=FloatSlider(min=l1-l3, max=l1+l3, step=0.1, value=0));

interactive(children=(FloatSlider(value=0.0, description='x', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…